Source: ISLR

In this notebook, I will implement *Boosting* based on decision trees, specifically Gradient Boosting and `XGBoost`. Similar to *Bagging*, *Boosting* is a general approach that can be applied to many machine learning methods. Trees in Boosting are built **sequentially**: each tree is grown using info from previously grown trees. Boosting, notably, does not involve bootstrap sampling and instead is fit on a modified version of the original data set.

Boosting learns *slowly*. Given the current model, we fit a decision tree to the residuals from the model instead of $Y$ (hospital charges). When then add this new decision tree into the fitted function in order to update the residuals. By fitting small trees to the residuals, we slowly improve $\hat{f}$ in areas where it does not perform well. The shrinkage parameter $\lambda$ slows the process down further, allowing more and different shaped trees to attack the residuals.

**Algorithm:**:

1. Set $\hat{f}(x) = 0$ and $r_i = y_i$, $\forall i $ in the training set 
2. For $b = 1, 2, \dots, B$, repeat:
    1. Fit a tree $\hat{f}^b$ with $d$ splits ($d+1$ terminal nodes) to the training data ($X,r$)
    2. Update $\hat{f}$ by adding in a shrunken version of the new tree: $\hat{f}(x) \leftarrow \hat{f}(x) + \lambda \hat{f}^b(x)$
    3. Update the residuals: $r_i \leftarrow r_i - \lambda  \hat{f}^b(x_i)$
    4. Output the boosted model: $\hat{f}(x) = \sum_{b=1}^{B} \lambda  \hat{f}^b(x)$
    
Boosting here has three hyperparameters (tuning parameters):
1. The number of trees $B$. Unlike *bagging* and *random forests*, boosting can overfit if $B$ is too large. We will use cross-validation to select $B$
2. The shrinkage parameter $\lambda$, a small positive number. This controls the rate at which boosting learns. Typical values are 0.01 or 0.001. Very small $\lambda$ will require large number of trees $B$
3. The number of $d$ splits in each tree which controls the complexity of the boosted ensemble. $d=1$ often wrosk well in which each tree is a *stump*, consisting of a single split. $d$ is also the *interaction depth* and controls the interaction order of the boosted model since $d$ splits can involve at most $d$ variables

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

import matplotlib.pyplot as plt

In [2]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Importing Dataset

In [4]:
# Importing processed dataframe
path_project = "C:/Users/Conno/Documents/Career/Projects/Hospital_Charges/tree_based_models"

os.chdir(path_project)
plots_dir = 'boosting_plots' # stores plots in plot folder

df = pd.read_csv("../df_processed.csv")

# do this later in data_cleaning file
bool_cols = df.select_dtypes(include=['bool']).columns
df[bool_cols] = df[bool_cols].astype(int)

#df.dtypes

## Train-Test Split

In [5]:
X = df.drop(columns = ["charges"])
y = df["charges"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 32)